# 1. Install and Import Dependencies

In [1]:
!pip install torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1 -f https://download.pytorch.org/whl/torch_stable.html


Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
     ---------------------------------------- 0.1/5.7 MB 1.3 MB/s eta 0:00:05
      --------------------------------------- 0.1/5.7 MB 980.4 kB/s eta 0:00:06
     - -------------------------------------- 0.1/5.7 MB 1.2 MB/s eta 0:00:05
     - -------------------------------------- 0.2/5.7 MB 1.1 MB/s eta 0:00:05
     - -------------------------------------- 0.3/5.7 MB 1.2 MB/s eta 0:00:05
     -- ------------------------------------- 0.3/5.7 MB 1.2 MB/s eta 0:00:05
     -- ------------------------------------- 0.4/5.7 MB 1.2 MB/s eta 0:00:05
     --- ------------------------------------ 0.4/5.7 MB 1.2 MB/s eta 0:00:05
     --- ------------------------------------ 0.5/5.7 MB 1.2 MB/s eta 0:00:05
     --- -----

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\cherub\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\cherub\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\cherub\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\urllib3\response.py", line 527, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "C:\Users\cherub\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
                  ^^^^^^^^^^^^^^^^^^^
  File "C:\Users\cherub\AppData\Local\Programs\Python\Python312\Lib\http\client.py", line 472, in read
    s = self.fp

     ---------------------------------------- 0.0/2.5 GB 1.9 MB/s eta 0:21:12
     ---------------------------------------- 0.0/2.5 GB 1.9 MB/s eta 0:21:12
     ---------------------------------------- 0.0/2.5 GB 1.8 MB/s eta 0:22:18
     ---------------------------------------- 0.0/2.5 GB 1.8 MB/s eta 0:22:18
     ---------------------------------------- 0.0/2.5 GB 1.8 MB/s eta 0:22:18
     ---------------------------------------- 0.0/2.5 GB 1.8 MB/s eta 0:22:44
     ---------------------------------------- 0.0/2.5 GB 1.8 MB/s eta 0:22:40
     ---------------------------------------- 0.0/2.5 GB 1.8 MB/s eta 0:22:40
     ---------------------------------------- 0.0/2.5 GB 1.8 MB/s eta 0:22:37
     ---------------------------------------- 0.0/2.5 GB 1.8 MB/s eta 0:22:40
     ---------------------------------------- 0.0/2.5 GB 1.8 MB/s eta 0:22:36
     ---------------------------------------- 0.0/2.5 GB 1.8 MB/s eta 0:22:40
     ---------------------------------------- 0.0/2.5 GB 1.8 MB/

In [3]:
!pip install transformers requests beautifulsoup4 pandas numpy

In [7]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

# 2. Instantiate Model

In [9]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

# 3. Encode and Calculate Sentiment

In [10]:
tokens = tokenizer.encode('It was good but couldve been better. Great', return_tensors='pt')

In [12]:
#tokenizer.decode(tokens[0]) ## to convert back to sentence
tokens

tensor([[  101, 10197, 10140, 12050, 10502, 12296, 10598, 10662, 16197,   119,
         11838,   102]])

In [13]:
result = model(tokens)

In [14]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [15]:
result.logits #output represents the probability of text being between 1-5, ranging from bad - good

tensor([[-2.7768, -1.2353,  1.4419,  1.9804,  0.4584]],
       grad_fn=<AddmmBackward0>)

In [16]:
int(torch.argmax(result.logits))+1 ##converting our maximum probable sentiment score to an integer from 1 to 5

4

# 4. Collect Reviews

In [17]:
## WEB SCRAPPIN
r = requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont') # to grab web page
soup = BeautifulSoup(r.text, 'html.parser') # passing the text from webpage as r.text give into the soup and parsed with html parser
regex = re.compile('.*comment.*') # look for onl yanything that has comment within the class
results = soup.find_all('p', {'class':regex}) # find all paragraphs 'p' in a class that starts with a comment
reviews = [result.text for result in results] #grabbing only the comment text from results, one by one using list comprehension

In [19]:
reviews

['The food was delicious. The ricotta pancakes were light and tasty.  The cream brûlée French toast was a fruit lovers delight. There was fresh strawberries, blueberries, passion fruit and custard. No syrup required. And crispy kale as a side ordered with eggs. Worth the walk over the Pyrmont bridge.',
 "Very cute coffee shop and restaurant. They have a lovely outdoor seating area and several tables inside.  It was fairly busy on a Tuesday morning but we were to grab the last open table. The server was so enjoyable, she chatted and joked with us and provided fast service with our ordering, drinks and meals. The food was very good. We ordered a wide variety and every meal was good to delicious. The sweet potato fries on the Chicken Burger plate were absolutely delicious, some of the best I've ever had. I definitely enjoyed this cafe, the outdoor seating, the service and the food!!",
 'Not only was the coffee delicious, but the service at Social Brew Cafe was exceptional. The staff were 

# 5. Load Reviews into DataFrame and Score

In [20]:
import numpy as np
import pandas as pd

In [31]:
df = pd.DataFrame(np.array(reviews), columns=['review']) ##loading reviews in dataframe. creating dataframe

In [29]:
df['review'].iloc[0] ##picking out the first review

'The food was delicious. The ricotta pancakes were light and tasty.  The cream brûlée French toast was a fruit lovers delight. There was fresh strawberries, blueberries, passion fruit and custard. No syrup required. And crispy kale as a side ordered with eggs. Worth the walk over the Pyrmont bridge.'

In [32]:
df

,review
0,The food was delicious. The ricotta pancakes w...
1,Very cute coffee shop and restaurant. They hav...
2,"Not only was the coffee delicious, but the ser..."
3,Social Brew Cafe exceeded expectations with th...
4,Six of us met here for breakfast before our wa...
5,Great place with delicious food and friendly s...
6,"Great service, lovely location, and really ama..."
7,Great food amazing coffee and tea. Short walk ...
8,Ricotta hot cakes! These were so yummy. I ate ...
9,It was ok. Had coffee with my friends. I'm new...


In [36]:
# function to get sentiment analysis
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [37]:
sentiment_score(df['review'].iloc[1])  #passing second review into fucn for classification

5

In [38]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x)) ##looping through all reviews and passing them into the function to be clasified. 512 represents the limit of tokens the model has
##using the .apply(lamba) func where x is all instance of reviews being passed as argument in the funct to be classified, but from 1:512 (sentiment_score(x[:512])

In [39]:
df ## calling the dataframe to shoe reviews and classification

,review,sentiment
0,The food was delicious. The ricotta pancakes w...,4
1,Very cute coffee shop and restaurant. They hav...,5
2,"Not only was the coffee delicious, but the ser...",5
3,Social Brew Cafe exceeded expectations with th...,5
4,Six of us met here for breakfast before our wa...,4
5,Great place with delicious food and friendly s...,5
6,"Great service, lovely location, and really ama...",5
7,Great food amazing coffee and tea. Short walk ...,5
8,Ricotta hot cakes! These were so yummy. I ate ...,5
9,It was ok. Had coffee with my friends. I'm new...,3


In [ ]:
df['review'].iloc[3]